<a href="https://colab.research.google.com/github/lucila-03/ML/blob/main/MVP_Imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Definição do Problema *

### Classificador de imagens binária -Tartaruga x Pinguim

Dataset do kaggle, separado em arquivos, é um problema de visão computacional para destinguir a imagem é uma tartaruga ou pinguim. O sistema deve indentificar e classificar a imagem conforme modelos de machine learning.

In [93]:
# Importação dos pacotes
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.metrics as skm
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from IPython.display import Image, display
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import numpy as np
import matplotlib.pyplot as plt

#### Utilidades

`plot_confusion_matrix` é uma função python que imprime uma matriz de confusão

In [94]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print('Matriz de confusão sem normalização')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label real')
    plt.xlabel('Label predito')

### 2. Carga de Dados

In [95]:
train_dir = 'ML/train'
val_dir = 'ML/valid/valid'
test_dir = 'ML/test'



Definação do tamanho do batch e as dimensões das imagens

In [96]:
batch_size = 32
img_height = 224
img_width = 224
num_classes = 2


Definação dos caminhos para os dados de treinamento e teste

Definação do tamanho do batch e as dimensões das imagens

In [97]:
batch_size = 32
img_height = 224
img_width = 224
num_classes = 2

Preparação dos dados

In [98]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Criação dos data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical') #training set

val_generator = test_datagen.flow_from_directory(val_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical')

FileNotFoundError: ignored

Agara vamos mover todos os arquivos imagem de carros e SUVs para pastas principais train e valid (Treino e validação)

Faço uma listagem dos arquivos que estão nas pastas train e valid

In [ ]:
#Lista 20 imagens aleatorias de cada diretorio de treino e validação

import os
import random

diretorio_Pinguim = './mvp/train/Pinguim'
diretorio_Tartaruga = './mvp/train/Tartaruga'
diretorio_valid = './mvp/train/valid'
num_amostras = 5

print(f"Amostras das imagens do diretório '{diretorio_train/diretorio_Pinguim}':")
print(f"Amostras das imagens do diretório '{diretorio_train/diretorio_Tartaruga}':")
print(f"Amostras das imagens do diretório '{diretorio_valid}':")
for imagem in imagens_amostra_valid:
    print(imagem)


#### Verificando a quantidade de imagens das duas classes no diretorio de treino e validação


In [ ]:
#verificando as quantidade de imagens de treino e de validação
import os

diretorio_base = './mvp'

def contar_imagens(diretorio):
    total_imagens = 0
    for raiz, diretorios, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            if arquivo.lower().endswith(('.jpeg', '.jpg')):
                total_imagens += 1
    return total_imagens

# Percorrer os diretórios em ./mvp
for diretorio in os.listdir(diretorio_base):
    diretorio_path = os.path.join(diretorio_base, diretorio)
    if os.path.isdir(diretorio_path):
        total = contar_imagens(diretorio_path)
        print(f"Total de imagens em {diretorio}: {total}")

Criando dataframes com as informações das imagens para train e valid

In [ ]:
#Criando o Dataframe com as imagens de treino e teste

import os
import pandas as pd

diretorio_train = './mvp/train'
diretorio_valid = './mvp/valid'

def criar_dataframe(diretorio):
    dados = []
    for raiz, diretorios, arquivos in os.walk(diretorio):
        for nome_arquivo in arquivos:
            if nome_arquivo.lower().endswith(('.jpeg', '.jpg')):
                caminho_imagem = os.path.join(raiz, nome_arquivo)
                dados.append({'Diretório': raiz, 'Imagem': nome_arquivo})
    return pd.DataFrame(dados)

df_train = criar_dataframe(diretorio_train)
df_valid = criar_dataframe(diretorio_valid)

print("Dataframe com as imagens do diretório 'train':")
display(df_train)

print("\nDataframe com as imagens do diretório 'valid':")
display(df_valid)


Criando os atributos necessários para desenvolver o trabalho

In [ ]:
#Criando os Atributos de Largura, Altura, Imagem, Canais, Classe
import os
import pandas as pd
from PIL import Image

diretorio_train = './mvp/train'
diretorio_valid = './mvp/valid'

dados_imagens_train = []
dados_imagens_valid = []

def processar_imagens(diretorio, dados_imagens):
    for raiz, diretorios, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            if arquivo.lower().endswith(('.jpeg', '.jpg')):
                caminho_arquivo = os.path.join(raiz, arquivo)
                with Image.open(caminho_arquivo) as img:
                    largura, altura = img.size
                    canais = img.mode
                    classe = 'Car' if 'car' in arquivo.lower() else 'Truck'
                    dados_imagens.append({'Imagem': arquivo, 'Largura': largura, 'Altura': altura, 'Canais': canais, 'Classe': classe})

processar_imagens(diretorio_train, dados_imagens_train)
processar_imagens(diretorio_valid, dados_imagens_valid)

df_train = pd.DataFrame(dados_imagens_train)
df_valid = pd.DataFrame(dados_imagens_valid)

print('DataFrames com os campos adicionados')


Visualização dos datasets com os atributos criados

In [ ]:
print("Dataframe com as imagens do diretório de treinamento:")
display(df_train)

print("\nDataframe com as imagens do diretório de validação:")
display(df_valid)

Plotando um gráfico de barras empilhado com o numero de imagens por classe.

In [ ]:
# Contagem das classes
contagem_classes_train = df_train['Classe'].value_counts()
contagem_classes_valid = df_valid['Classe'].value_counts()

# Plotagem do gráfico de barras
fig, ax = plt.subplots()
bar_plot_train = contagem_classes_train.plot(kind='bar', ax=ax, color='blue', label='Train')
bar_plot_valid = contagem_classes_valid.plot(kind='bar', ax=ax, color='orange', label='Valid')

# Configurações do gráfico
ax.set_title('Cars vs Trucks - Dados Disponíveis')
ax.set_xlabel('Classe')
ax.set_ylabel('Contagem')

# Configuração da legenda
ax.legend()

# Exibição do gráfico
plt.show()

Visualizando algumas imagens do conjunto de dados de treino

In [ ]:

#setando o diretorio onde se encontram as imagens
diretorio_train = './mvp/train'
#Setando o numero de imagens  serem exibidas
num_imagens = 20
#determinando o tamanho da imagem
tamanho_imagem = (128, 128)

# Selecionar aleatoriamente 20 índices de imagens do df_train
indices_aleatorios = random.sample(range(len(df_train)), num_imagens)

# Configurar a figura para exibir as imagens
fig, axs = plt.subplots(4, 5, figsize=(12, 8))
fig.subplots_adjust(hspace=0.4)

# Plotar as imagens selecionadas aleatoriamente
for i, indice in enumerate(indices_aleatorios):
    imagem = df_train.loc[indice, 'Imagem']
    caminho_imagem = os.path.join(diretorio_train, imagem)
    img = Image.open(caminho_imagem)
    img = img.resize(tamanho_imagem)

    linha = i // 5
    coluna = i % 5

    axs[linha, coluna].imshow(img)
    axs[linha, coluna].set_title(df_train.loc[indice, 'Classe'])
    axs[linha, coluna].axis('off')

plt.show()


Divisão dos dados em treino e teste: 30% das imagens para teste e 70% para treinamento do modelo de deep learning. Importante lembrar que os conjuntos devem ser disjuntos, ou seja, não devem poussir nenhum elemento em comum

In [ ]:
from sklearn.model_selection import train_test_split

# Definir os campos a serem usados para a separação
features = ['Imagem', 'Classe']

# Separar os conjuntos de treino e teste
train, test = train_test_split(df_train[features], test_size=0.3, random_state=42)

Visualizando a divisão dos dados de treino e de teste

In [ ]:
# Exibir informações sobre os conjuntos de treino e teste
print("Conjunto de Treino:")
display(train)
print("")

print("Conjunto de Teste:")
display(test)

Vamos utilizaar uma técnica amplamente utilizada no campo de aprendizado de máquina e visão computacional para aumentar a diversidade e a quantidade de dados de treinamento, aplicando transformações às imagens existentes. A técnica utilizada é o data augmentation, sob demanda durante o treinamento do modelo.


In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
diretorio_train = './mvp/train'
diretorio_valid = './mvp/valid'
target_size = (128, 128) # Dimensões desejadas das imagens (largura, altura)
batch_size = 32  # Tamanho do lote de dados

# Configurar o ImageDataGenerator com split de validação e parâmetros solicitados
print('Configurando o ImageDataGenerator')
train_datagen = ImageDataGenerator(validation_split=0.2,
                             rescale=1.0/255,
                             rotation_range=48,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

print('---------------------------')

print('Gerando dados teste')
test_datagen=ImageDataGenerator(rescale=1./255)
print('---------------------------- ')
# Gerar os dados de treino
print('Gerando dados de treino')
train_generator=train_datagen.flow_from_dataframe(
    train ,diretorio_train,
    target_size=(128,128),
    batch_size=50,
    class_mode='binary',
    x_col='Imagem',
    y_col='Classe',
    subset='training' #training set
    )

print('---------------------------')


# Gerar os dados de validação
print('Gerando dados de validação')
val_generator=train_datagen.flow_from_dataframe(
    train ,diretorio_train,
    target_size=(128,128),
    batch_size=50,
    class_mode='binary',
    x_col='Imagem',
    y_col='Classe',
    subset='validation' #validation  set
    )
print('------------------------------')

# Gerar os dados de teste

print('Gerando dados de Teste')

test_generator=train_datagen.flow_from_dataframe(
    test ,diretorio_train,
    target_size=(128,128),
    batch_size=50,
    class_mode='binary',
    x_col='Imagem',
    y_col='Classe',
    subset='training') #teste

print('------------------------------')

Visualizando um exemplo do tratamento das imagens

In [ ]:

# Pegando um registro do dataframe
sample = df_train.sample(n=1)

# Neste caso o tipo de classe é categorico, pois temos um exemplo com apenas uma classe
sample_generator = train_datagen.flow_from_dataframe(
    sample,
    diretorio_train,
    x_col='Imagem',
    y_col='Classe',
    target_size=(128,128),
    class_mode='categorical'
)

plt.figure(figsize=(12, 9))
for i in range(9):
    plt.subplot(3, 3, i+1)
    for x_batch, y_batch in sample_generator:
        Imagem = x_batch[0]
        plt.imshow(Imagem)
        plt.axis('Off')
        break
plt.tight_layout()
plt.show()

###Configuração de um modelo de deep learning usando uma rede neural convolucional (CNN) simples com a biblioteca Keras

#### Definição da arquitetura do modelo

Aqui é especificada uma `camada convolucional 2D` que possui 32 filtros, um tamanho de filtro de 3x3 que usa a função de ativação `ReLU`. O parâmetro `input_shape` define a forma das imagens de entrada para a rede. Na sequência são adicionadas camadas `max-pooling` com a mesma função de ativação.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Crie o modelo da CNN
model = Sequential()

# Adicione as camadas convolucionais
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten
model.add(Flatten())

# Adicione camadas densas
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Para um problema de classificação binária

model.summary()

###Treinamento do modelo de deep learning

#### Compilação do modelo Keras com as configurações do treinamento

`optimizer='adam'`: especifica o otimizador a ser usado durante o treinamento.

`loss='binary_crossentropy'`: especifica a função de perda a ser usada durante o treinamento. Para problemas de classificação binária, onde a variável alvo tem apenas duas categorias, a perda binária de entropia cruzada é freqüentemente empregada.

`metrics=['accuracy']`: especifica as métricas de avaliação a serem usadas durante o treinamento e o teste.

In [ ]:
# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

O método fit() inicia o processo de treinamento do modelo usando o gerador de dados para o treinamento train_generator e o gerador de dados de validação val_generator.
Criado uma função para guardar as métricas de treinamento e validação.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback

# Criar um callback personalizado para salvar as métricas de treinamento e validação
class MetricsCallback(Callback):
    def __init__(self):
        self.train_accuracy = []
        self.val_accuracy = []
        self.train_loss = []
        self.val_loss = []

    def on_epoch_end(self, epoch, logs=None):
        self.train_accuracy.append(logs['accuracy'])
        self.val_accuracy.append(logs['val_accuracy'])
        self.train_loss.append(logs['loss'])
        self.val_loss.append(logs['val_loss'])

# Criar uma instância do callback personalizado
metrics_callback = MetricsCallback()

# Definir número de épocas e steps por época
num_epochs = 100
steps_per_epoch = 58

# Treinar o modelo usando o callback e os parâmetros definidos
model.fit_generator(train_generator, validation_data=val_generator, epochs=num_epochs, steps_per_epoch=steps_per_epoch, callbacks=[metrics_callback])

# Avaliar o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_generator)

# Obter as métricas do callback personalizado
train_accuracy = metrics_callback.train_accuracy
val_accuracy = metrics_callback.val_accuracy
train_loss = metrics_callback.train_loss
val_loss = metrics_callback.val_loss

# Criar um array com o número de épocas
epochs = range(1, len(train_accuracy) + 1)

# Plotar a acurácia
plt.plot(epochs, train_accuracy, 'bo-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'ro-', label='Validation Accuracy')
plt.axhline(test_accuracy, color='g', linestyle='-', label='Test Accuracy')
plt.title('Training, Validation, and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotar a perda
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.axhline(test_loss, color='g', linestyle='-', label='Test Loss')
plt.title('Training, Validation, and Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


###Carregando o modelo

Carregando o modelo salvo para ganho de tempo na correção.

In [ ]:
import requests
import os
import keras

# URL do modelo
url_model = 'http://liessin.com.br/mvp/sprint2/trained_model.h5'

# Caminho local para salvar o modelo
local_path = 'trained_model.h5'

# Baixar o modelo do URL e salvá-lo localmente
response = requests.get(url_model)
with open(local_path, 'wb') as f:
    f.write(response.content)

# Carregar o modelo
loaded_model = keras.models.load_model(local_path)
print("Modelo carregado com sucesso.",local_path)

Inicialmente, a acurácia estava em 0,6136, o que significa que o modelo estava classificando corretamente cerca de 61% dos exemplos. No entanto, após o treinamento, a acurácia aumentou para 0,9560, indicando que o modelo melhorou e agora classifica corretamente cerca de 96% dos exemplos com dos dados de treino.

Da mesma forma, a perda inicial era de 0,6373, e após o treinamento, a perda diminuiu para 0,1030. Uma perda menor indica que o modelo está fazendo previsões mais precisas e se ajustando melhor aos dados de treino.

Portanto, podemos concluir que o treinamento do modelo CNN com 100 épocas foi eficaz, pois resultou em uma melhoria significativa na acurácia e uma redução na perda. Isso indica que o modelo foi capaz de aprender e se ajustar aos dados de treinamento, fornecendo previsões mais precisas.

Com os dados de teste obtivemos uma perda de  0.4562  e uma acurácia de 0.8666 ou seja 87% dos dados.

###Avaliação do modelo de deep learning

Acurácia do modelo nos dados de teste

In [ ]:
# Avaliar o modelo nos dados de teste
test_loss, test_accuracy = model.evaluate(test_generator)

# Imprimir a perda e        a acurácia
print("Perda nos dados de teste: {:.4f}".format(test_loss))
print("Acurácia nos dados de teste: {:.2f}%".format(test_accuracy * 100))

###Execução do modelo treinado em um subconjunto das imagems de teste
Idealmente se deseja realizar a inferência sobre todo o conjunto de teste, porém para efeitos de agilidade, executamos apenas em um subconjunto de imagens.

In [ ]:
count = 0
y_pred = list()
y_true = list()
class_names = ['Car', 'Truck']

# Loop do data generator de teste que contem as imagens
for batch in test_generator:

    count+=1

    # Condição de saída do loop. Só executaremos o modelo em algumas imagens
    if count == 2:
        break

    else:

        # Pegando a imagem e o label
        images = batch[0]
        labels = batch[1]

        # Previsão
        predictions = model.predict(images)

        # Visualização da imagem
        for i in range(len(images)):

            image = images[i]
            label = labels[i]
            prediction = predictions[i]

            # Converte o array de previsão para visualizar a classe e a probabilidade
            predicted_class = np.argmax(prediction)
            probability = prediction[predicted_class]
            label = class_names[int(label)]
            y_true.append(label)

            # Faz o DE-PARA da probabilidade com o tipo de classe da previsão
            class_name = 'Car' if prediction >= 0.5 else 'Truck'
            y_pred.append(class_name)

            # Configura a saída das imagens que serão visualizadas
            plt.imshow(images[i])
            plt.axis('Off')
            plt.show()

            # Imprime os resultados da classificação das images
            print("Label: ", label)
            print("Previsão: ", class_name)
            print("Probabilidade: ", probability)

In [ ]:
print(y_pred)
print(y_true)


In [ ]:

def test_model(model, test_generator):
    count = 0
    y_pred = []
    y_true = []
    class_names = ['Car', 'Truck']
    contador = 1


    # Loop do data generator de teste que contem as imagens
    for batch in test_generator:
        # Pegando a imagem e o label
        images = batch[0]
        labels = batch[1]

        # Previsão
        predictions = model.predict(images)

        # Visualização da imagem
        for i in range(len(images)):

            imagem = images[i]
            label = labels[i]
            prediction = predictions[i]

            # Converte o array de previsão para visualizar a classe e a probabilidade
            predicted_class = np.argmax(prediction)
            probability = prediction[predicted_class]
            label = class_names[int(label)]
            y_true.append(label)

            # Faz o DE-PARA da probabilidade com o tipo de classe da previsão
            class_name = 'Truck' if prediction >= 0.5 else 'Car'
            y_pred.append(class_name)

            # Configura a saída das imagens que serão visualizadas
            plt.imshow(imagem)
            plt.axis('off')
            plt.show()
            # Imprime os resultados da classificação das imagens
            print("Label: ", label)
            print("Previsão: ", class_name)
            print("Probabilidade: ", probability)
            print('Número da Imagem: ', contador)
            contador += 1





# Chamar a função para testar o modelo
test_model(model, test_generator)


In [ ]:
print(y_pred.value_counts)

### Visualização de métricas da avaliação do modelo

Métricas de performance do modelo no subconjunto das imagens de teste

In [ ]:
accuracy = skm.accuracy_score(y_true, y_pred)
precision = skm.precision_score(y_true, y_pred, average='weighted')
recall = skm.recall_score(y_true, y_pred, average='weighted')
f1score = skm.f1_score(y_true, y_pred, average='weighted')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1score)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Fazer as previsões no subconjunto de imagens de teste
y_pred = model.predict(test_generator)

y_pred = np.argmax(y_pred, axis=1)  # Converte as previsões para as classes preditas

# Obter os rótulos verdadeiros do subconjunto de imagens de teste
y_true = test_generator.classes

# Calcular a matriz de confusão
confusion = confusion_matrix(y_true, y_pred)
class_names = ['Car', 'Truck']

# Calcular as métricas de classificação (precisão, recall, F1-score)
classification = classification_report(y_true, y_pred, target_names=class_names)

# Imprimir a matriz de confusão e as métricas de classificação
print("Matriz de Confusão:")
print(confusion)
print("\nMétricas de Classificação:")
print(classification)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Fazer as previsões no subconjunto de imagens de teste
y_pred = model.predict(test_generator)

# Converter as previsões para as classes preditas
#y_pred = np.argmax(y_pred, axis=1)

# Aplicar a regra de atribuição de 0 ou 1
for i in range(len(y_pred)):
    if y_pred[i] < 0.5:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

# Obter os rótulos verdadeiros do subconjunto de imagens de teste
y_true = test_generator.classes

# Calcular a matriz de confusão
confusion = confusion_matrix(y_true, y_pred)
class_names = ['Car', 'Truck']

# Calcular as métricas de classificação (precisão, recall, F1-score)
classification = classification_report(y_true, y_pred, target_names=class_names)

# Imprimir a matriz de confusão e as métricas de classificação
print("Matriz de Confusão:")
print(confusion)
print("\nMétricas de Classificação:")
print(classification)


In [ ]:
set(y_pred)

Matriz de confusão para identificar onde o modelo de deep learning acertou e errou na classificação do subconjunto de imagens de teste

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Fazer as previsões no subconjunto de imagens de teste
y_pred = model.predict(test_generator)
#y_pred = np.argmax(y_pred, axis=1)  # Converte as previsões para as classes preditas

# Aplicar a regra de atribuição de 0 ou 1
for i in range(len(y_pred)):
    if y_pred[i] < 0.5:
        y_pred[i] = 0
    else:
        y_pred[i] = 1
# Obter os rótulos verdadeiros do subconjunto de imagens de teste
y_true = test_generator.classes

# Calcular a matriz de confusão
confusion = confusion_matrix(y_true, y_pred)

# Criar um heatmap da matriz de confusão
sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")

# Definir rótulos dos eixos
plt.xlabel('Classe Predita')
plt.ylabel('Classe Verdadeira')

# Definir título do gráfico
plt.title('Matriz de Confusão')

# Exibir o gráfico
plt.show()


###Exportação do modelo de deep learning para posterior uso

Salvando o modelo de deep learning que foi treinado

In [ ]:
# obtendo a data e hora atual
now = datetime.now()

# Definição do formato
format = '%Y-%m-%dT%H%M'

# Converter a data e hora em uma string com o formato especificado
formatted_datetime = now.strftime(format)

path_model = 'mvp/trained_models'

name_model = 'trained_model_' + formatted_datetime + '.h5'

# salvando o modelo
model.save("%s/%s" % (path_model, name_model))
print("Modelo salvo com o nome: ", name_model)

In [ ]:
count = 0
y_pred = list()
y_true = list()
class_names = ['Car', 'Truck']
Imagem=0
# Loop do data generator de teste que contem as imagens
for batch in test_generator:

    count+=1

    # Condição de saída do loop. Só executaremos o modelo em algumas imagens
    if count == 2:
        break

    else:

        # Pegando a imagem e o label
        images = batch[0]
        labels = batch[1]

        # Previsão
        predictions = loaded_model.predict(images)

        # Visualização da imagem
        for i in range(len(images)):
            Imagem +=1
            image = images[i]
            label = labels[i]
            prediction = predictions[i]

            # Converte o array de previsão para visualizar a classe e a probabilidade
            predicted_class = np.argmax(prediction)
            probability = prediction[predicted_class]
            label = class_names[int(label)]
            y_true.append(label)

            # Faz o DE-PARA da probabilidade com o tipo de classe da previsão
            class_name = 'Car' if prediction >= 0.5 else 'Truck'
            y_pred.append(class_name)

            # Configura a saída das imagens que serão visualizadas
            plt.imshow(images[i])
            plt.axis('Off')
            plt.show()

            # Imprime os resultados da classificação das images
            print("Label: ", label)
            print("Previsão: ", class_name)
            print("Probabilidade: ", probability)
            print("Imagem: ", Imagem)